In [1]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, KFold
from xgboost import DMatrix


### Input Data - Cleaned Dataset All

In [2]:

# df_TagDesc = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\CSV\Not for Processing\TagDesc.csv')

# df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
# df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# # Concatenate (union) the dataframes
# df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

df_All = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Testing Data\Merge Data\merged_data.csv')

print(df_All.head())


                  Date  425_pct_Al    Al2O3  M_Value  C4_pct_Eth  C4_pct_H2O  \
0  2012-05-16 16:00:00    6.307630  11.3300  3.50893    2.508420    21.96340   
1  2012-05-16 18:00:00    6.308335  11.3750  3.50863    2.511135    21.96865   
2  2012-05-16 19:00:00    6.309390  11.4425  3.50818    2.515210    21.97655   
3  2012-05-16 20:00:00    6.310090  11.4875  3.50788    2.517925    21.98185   
4  2012-05-16 21:00:00    6.310790  11.5325  3.50758    2.520640    21.98715   

   HydWtr_pct_Ammonia  C4_pct_Hex  HydWtr_Na2O    Butanol  ...    FC55576  \
0            0.908765    0.659256     2.712180  54.858300  ...  334.94800   
1            0.907643    0.659930     2.805415  54.858300  ...  334.94800   
2            0.905959    0.660942     2.945270  54.858300  ...  334.94800   
3            0.904837    0.661616     3.038505  51.190050  ...  332.47200   
4            0.903715    0.662291     3.131740  48.744567  ...  338.63375   

   FFC55553  FFC55555    LC55557    LC90366    LC90368  

In [3]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [4]:
# # # List of columns to exclude from the XGboost feature selection results
# exclude_columns = [
#                     'TI40050',
#                     'TC55555',
#                     'LC55568',
#                     'LC55557',
#                     'Al2O3',
#                     'LC55553',
#                     'M_Value',
#                     'FFC55553',
#                     'DI55580',
#                     'Al2O3',
#                     'C4_pct_Hex',
#                     'LC52572',

# #                  'Date',
# #                  'C4_pct_Hex', 'HydWtr_Na2O',
# #                  'TC55555'  
#                    ]

# # # Create a new DataFrame without the excluded columnsd
# df_All = df_All.drop(columns=exclude_columns)

In [5]:
df_All.columns

Index(['Date', '425_pct_Al', 'Al2O3', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O', 'Butanol', 'Decanol',
       'DI55102', 'DI55152', 'DI55580', 'FC55003', 'FC55009', 'FC55552',
       'FC55569', 'FC55576', 'FFC55553', 'FFC55555', 'LC55557', 'LC90366',
       'LC90368', 'PI55020', 'TC55552', 'TI55021', 'Year'],
      dtype='object')

In [6]:
# df_All = df_All[df_All['Date'] > '2020-06-15 00:00:00']

In [7]:
# df_All.drop('Date', axis=1, inplace=True)




In [8]:
print(df_All.describe())

         425_pct_Al         Al2O3       M_Value    C4_pct_Eth    C4_pct_H2O  \
count  47714.000000  47714.000000  47714.000000  47714.000000  47714.000000   
mean       6.148541     11.355834      3.592181      1.210588     21.161871   
std        0.255252      0.570302      0.177351      0.687390      2.756784   
min        5.174085      9.531050      1.347070      0.064828      8.149630   
25%        6.006720     11.084600      3.500671      0.627316     18.776275   
50%        6.142755     11.483750      3.587543      1.107550     21.435250   
75%        6.290847     11.701963      3.679920      1.674442     23.312362   
max        7.134450     12.879900      5.827565     11.732650     34.572900   

       HydWtr_pct_Ammonia    C4_pct_Hex   HydWtr_Na2O       Butanol  \
count        47714.000000  47714.000000  47714.000000  47714.000000   
mean             0.970035      0.466893      0.807371      9.791394   
std              0.149562      0.241990      0.696261      9.707619   
min 

In [9]:
# # Assuming your data is in filtered_df and you want to predict 'target_column_name'
# df = df_All.drop(['Butanol', 'FC55009', 'PI55004', 'PI55020','TI55021'], axis=1)  # Replace 'target_column_name' with your target column's name
# # y = df_All['Decanol']

In [10]:
# # Keep only rows with specified IDs
# feature_names = [
# '425_pct_Al',
# 'C4_pct_Eth',
# 'C4_pct_H2O',
# 'HydWtr_pct_Ammonia',
# 'HydWtr_Na2O',
# 'DI55102',
# 'DI55152',
# 'FC55003',
# 'FC55009',
# 'FC55552',
# 'FC55569',
# 'FC55576',
# 'FFC55555',
# 'LC90366',
# 'LC90368',
# 'PI55004',
# 'PI55020',
# 'TC55552',
# 'TI55021'
# ]
    

In [11]:
import pickle

# Load the trained XGBoost models
with open(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C10\5 Preprocessing - Slecected Features\Updated Model\C10_xgb_model.sav', 'rb') as model_file:
    c10_model = pickle.load(model_file)
with open(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C4\5 Preprocessing - Slecected Features\Updated Model\C4_xgb_model.sav', 'rb') as model_file:
    c4_model = pickle.load(model_file)
with open(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model STEAM - FC55569\5 Preprocessing - Slecected Features\Updated Model\STEAM_xgb_model.sav', 'rb') as model_file:
    steam_model = pickle.load(model_file)

# Initialize the new columns in the original df
df_All['FFC55555_adj'] = df_All['FFC55555']
df_All['Decanol_adj'] = None  # Using None as a placeholder
df_All['Butanol_adj'] = None  # Using None as a placeholder
df_All['FC55569_adj'] = None  # Using None as a placeholder

def predict_value(model, row, feature_names):
    # Ensure 'row' is prepared correctly for your model
    data_for_prediction = row[feature_names].values.reshape(1, -1)
    prediction = model.predict(data_for_prediction)[0]
    return prediction

# Assume 'decanol_feature_names' and 'butanol_feature_names' are defined
# You should define these lists based on the features your models expect
decanol_feature_names = ['425_pct_Al', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O', 'Decanol', 'FC55003',
       'FC55009', 'FC55552', 'FC55569', 'FFC55553', 'FFC55555', 'LC90366',
       'LC90368', 'PI55020', 'TC55552', 'TI55021']  
butanol_feature_names = ['425_pct_Al', 'C4_pct_Eth', 'C4_pct_H2O', 'HydWtr_pct_Ammonia',
       'C4_pct_Hex', 'HydWtr_Na2O', 'Butanol', 'DI55102', 'DI55152', 'FC55552',
       'FC55569', 'FC55576', 'FFC55553', 'FFC55555', 'LC55557', 'LC90366',
       'LC90368', 'TC55552']
steam_feature_names = ['425_pct_Al', 'Al2O3', 'C4_pct_Eth', 'C4_pct_H2O', 'C4_pct_Hex',
       'FC55569', 'DI55102', 'DI55152', 'DI55580', 'FC55003', 'FC55576',
       'FFC55555', 'LC90366', 'PI55020']

# Your prediction and adjustment loop
for index, row in df_All.iterrows():
    # Adjust row for prediction if necessary
    row_for_prediction = row.copy()
    row_for_prediction['FFC55555'] = row_for_prediction['FFC55555_adj']
    
    # Process record through model C10 for Decanol
    df_All.at[index, 'Decanol_adj'] = predict_value(c10_model, row_for_prediction, decanol_feature_names)
    
    # Process record through model C4 for Butanol
    df_All.at[index, 'Butanol_adj'] = predict_value(c4_model, row_for_prediction, butanol_feature_names)

    # Process record through Optimization Logic
    while True:
        decanol = df_All.at[index, 'Decanol_adj']
        butanol = df_All.at[index, 'Butanol_adj']
        ffc_adjusted = df_All.at[index, 'FFC55555_adj']

        if decanol < 5:
            if ffc_adjusted >= 0.66: # Can be adjusted to the lowest rC4 desired
                df_All.at[index, 'FFC55555_adj'] -= 0.01
            else:
                break  # Next Record
        else:
            while decanol >= 5:
                if df_All.at[index, 'FFC55555'] >= ffc_adjusted:
                    break  # Next Record
                else:
                    if ffc_adjusted <= 1.19: # Can be adjusted to the highest rC4 desired
                        df_All.at[index, 'FFC55555_adj'] += 0.01
                    else:
                        break
            break

        # Update predictions after adjustment
        row_for_prediction['FFC55555'] = df_All.at[index, 'FFC55555_adj']
        df_All.at[index, 'Decanol_adj'] = predict_value(c10_model, row_for_prediction, decanol_feature_names)
        

    # Update the FC55569_adj column with the output from the steam model
    df_All.at[index, 'Butanol_adj'] = predict_value(c4_model, row_for_prediction, butanol_feature_names)
    df_All.at[index, 'FC55569_adj'] = predict_value(steam_model, row_for_prediction, steam_feature_names)

ValueError: Feature shape mismatch, expected: 18, got 19

In [ ]:
df_All.head()